<a href="https://colab.research.google.com/github/ThanuMahee12/ayush-vision/blob/data-prepare/DataSet/Prepare/DataSetPrepareInGoogle_Account.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Colab

In [1]:
from google.colab import auth

In [1]:
from googleapiclient.discovery import build

In [3]:
from google.colab import drive

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
firebase_key='/content/drive/MyDrive/ServiceKey.json'

In [10]:
firebase_count=1

### user

In [7]:
auth.authenticate_user()
service = build('oauth2', 'v2')
user_info = service.userinfo().get().execute()
email = user_info['email']
profile=user_info['picture']


## Firebase

In [8]:
!pip install firebase-admin

In [6]:
import firebase_admin

In [7]:
from firebase_admin import credentials,storage


In [8]:
from firebase_admin import firestore

In [11]:

if firebase_count==1:
  cred = credentials.Certificate(firebase_key)
  firebase_admin.initialize_app(cred,{
    'storageBucket': 'ayush-vision-asw4gh.appspot.com'
})
firebase_count=firebase_count+1

In [12]:
db = firestore.client()

In [13]:
bucket = storage.bucket()

In [14]:
dataset_col = db.collection("dataset")

### Default

In [15]:
import numpy as np
import os
from datetime import date,datetime
import json
import csv
import yaml

In [16]:
def count_images_in_folder(folder_path):
    return sum([len(files) for r, d, files in os.walk(folder_path) if any(file.endswith(('.jpg', '.jpeg', '.png')) for file in files)])

def dictory_images_count(root_dir):
  train_images = count_images_in_folder(os.path.join(root_dir, 'train','images'))
  val_images = count_images_in_folder(os.path.join(root_dir, 'valid','images'))
  test_images = count_images_in_folder(os.path.join(root_dir, 'test','images'))
  total_images = train_images + val_images + test_images
  return {
      'total_count':total_images,
      'val_count':val_images,
      'train_count':train_images,
      'test_count':test_images
  }

## Yolo

In [17]:
Dataset_dir='/content/drive/MyDrive/AyushVision/yolo/dataset'

In [18]:
yolo_doc=dataset_col.document('yolo')
doc=yolo_doc.get()

In [19]:
if not doc.exists:
  doc_ref= db.collection("dataset").document('yolo')
  doc_ref.set({'algorithum':'yolo','version':'1','mode':'detection','url':'yolo'})
else:
  doc_ref= yolo_doc

#### Yolo Leaves

In [20]:
yolo_ref_leaves=doc_ref.collection("Leaves")

In [21]:
yolo_doc_leave_basic=yolo_ref_leaves.document('basic')

In [23]:
yolo_doc_leave_basic.set({
    'account':email,
    'profile':profile,
})

update_time {
  seconds: 1725862376
  nanos: 280581000
}

In [24]:
intial_basic_col=yolo_doc_leave_basic.collection("Intial")

In [22]:
subfolders=os.listdir(Dataset_dir)

In [23]:
total_datasetCount=0
names=[]

In [27]:
for sub in subfolders:
  yaml_file=os.path.join(Dataset_dir,sub,'data.yaml')
  sub_doc=intial_basic_col.document(sub)
  sub_doc_col_util=sub_doc.collection('Util')
  sub_doc_col_util_dataYaml=sub_doc_col_util.document('datYaml')
  sub_doc_col_util_Dataset_readMe=sub_doc_col_util.document('readme_Dataset')
  sub_doc_col_util_roboflow_readMe=sub_doc_col_util.document('readme_roboflow')
  dataset_count=dictory_images_count(os.path.join(Dataset_dir,sub))
  total_datasetCount=total_datasetCount+dataset_count['total_count']
  sub_doc.set(dataset_count)
  datasetLines,roboflowlines=[],[]
  with open(os.path.join(Dataset_dir,sub,'README.dataset.txt'), "r") as file:
    datasetLines = file.readlines()
  sub_doc_col_util_Dataset_readMe.set({'dataset':datasetLines})
  with open(os.path.join(Dataset_dir,sub,'README.roboflow.txt'), "r") as file:
    roboflowlines = file.readlines()
  sub_doc_col_util_roboflow_readMe.set({
      'roboflow':roboflowlines
  })
  with open(yaml_file, 'r') as file:
    yaml_content = yaml.safe_load(file)
    names.extend(yaml_content['names'])
    sub_doc_col_util_dataYaml.set(yaml_content)
    sub_doc.update({'names':yaml_content['names'],'no_of_species':yaml_content['nc'],'roboflow':yaml_content['roboflow']})

In [28]:
yolo_doc_leave_basic.update({
    'count':total_datasetCount,
    'names':list(set(names))
})

update_time {
  seconds: 1725862570
  nanos: 231251000
}

In [29]:
for subfold in subfolders:
  path=os.path.join(Dataset_dir,subfold)
  doc=intial_basic_col.document(subfold)
  species_col=doc.collection('Species')



### Prepare

In [24]:
intial_prepare_col=yolo_doc_leave_basic.collection("Prepare")

In [25]:
for sub in subfolders:
  yaml_file=os.path.join(Dataset_dir,sub,'data.yaml')
  sub_doc=intial_prepare_col.document(sub)
  sub_doc_col_util=sub_doc.collection('Util')
  sub_doc_col_util_dataYaml=sub_doc_col_util.document('datYaml')
  sub_doc_col_util_Dataset_readMe=sub_doc_col_util.document('readme_Dataset')
  sub_doc_col_util_roboflow_readMe=sub_doc_col_util.document('readme_roboflow')
  dataset_count=dictory_images_count(os.path.join(Dataset_dir,sub))
  total_datasetCount=total_datasetCount+dataset_count['total_count']
  sub_doc.set(dataset_count)
  datasetLines,roboflowlines=[],[]
  with open(os.path.join(Dataset_dir,sub,'README.dataset.txt'), "r") as file:
    datasetLines = file.readlines()
  sub_doc_col_util_Dataset_readMe.set({'dataset':datasetLines})
  with open(os.path.join(Dataset_dir,sub,'README.roboflow.txt'), "r") as file:
    roboflowlines = file.readlines()
  sub_doc_col_util_roboflow_readMe.set({
      'roboflow':roboflowlines
  })
  with open(yaml_file, 'r') as file:
    yaml_content = yaml.safe_load(file)
    names.extend(yaml_content['names'])
    sub_doc_col_util_dataYaml.set(yaml_content)
    sub_doc.update({'names':yaml_content['names'],'no_of_species':yaml_content['nc'],'roboflow':yaml_content['roboflow']})